# ECON202A Problem Set

Team members: David Johannes, Bailey Johnson, Chandni Raja, Natasha Watkins

### Problem 1

Given $u(c_t) = -\frac{1}{2} (\bar{c} - c_t)^2$ and $r = \delta$,

\begin{align*}
    u'(c_t) &= \bar{c} - c_t
\end{align*}

By the Theorem (Hall, pg. 974)

\begin{align*}
    E_t u'(c_{t+1}) &= \frac{1+\delta}{1+r} u'(c_t) \\
    \Rightarrow \bar{c} - E_t c_{t+1} &= \bar{c} - c_t \\
    c_t &= E_t c_{t+1} \\
    \Rightarrow c_{t+1} &= c_t - \varepsilon_{t+1}
\end{align*}

Where $\varepsilon_{t+1}$ is the true disturbance term, so that $E_t \varepsilon_{t+1} = 0$, and therefore is white noise. This is a random walk process with no constant. 

### Problem 2

The permanent-income hypothesis suggests that consumers' behaviour incorporates expectations of their future ability to consume, i.e., their consumption today should reflect their expectation of future income in all periods.  Forecasting future consumption should therefore only be a function of current consumption, as current consumption incorporates expectations of all other variables that might affect consumption, and so the stochastic process for income is irrelevant for forecasting.

As consumption in the next period is only a function of current consumption, the error term has no effect on the level of consumption, i.e., $E_t \varepsilon_{t+1} = 0$. Because $c_{t+1}$ is not explained by consumption in any other period, the error terms should be uncorrelated, i.e., $E_t(\varepsilon_{t+1} \varepsilon_{t+\tau}) = 0$ for all $\tau \neq 1$. Therefore, the error term is a white noise process.

### Problem 3

### Problem 4

In [ ]:
import pandas as pd
from fredapi import Fred
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.api import OLS, tsa
from numpy.linalg import inv

In [ ]:
fred = Fred(api_key='16fc433e0cb217bb8cb94bf76b981f2f')
c = fred.get_series('A794RX0Q048SBEA')  # Real personal consumption expenditures per capita

In [ ]:
c.plot(title='Real personal consumption expenditures per capita')
plt.show()

In [ ]:
data = pd.DataFrame()
data['y'] = np.log(c).values
data['X'] = np.log(c).shift(1).values # Shift to obtain consumption last period
data['constant'] = 1
data = data[1:]  # Drop first observation

In [ ]:
model = OLS(data.y, data[['constant', 'X']])
results = model.fit()
print(results.summary())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(data.index, results.fittedvalues, label='Fitted values')
ax.plot(data.y, label='Actual')
plt.legend()
plt.show()

In [ ]:
plt.plot(results.resid.index, results.resid)
plt.title('Residual plot')
plt.show()

The AR(1) process appears to be a reasonable statistical model of log consumption. The autoregressive coefficient is 0.999, indicating that log consumption is highly autoregressive. The residual plot shows that something similiar to a white noise process. The $R^2$ value indicates current consumption is explained almost perfectly by previous period's consumption.

### Problem 5

In [ ]:
ρ = 0.9
σ = np.sqrt(0.005)
θ = 0.25             # Capital share of income

We know from the properties of a Cobb-Douglas production function, the capital share of income is
$$
\frac{rk}{y} = \frac{rk}{k^\theta} = rk^{1 - \theta} = 0.25 = \theta
$$

To find $\beta$ and $\delta$, note that the share of investment is
$$
\frac{i}{y} = \frac{\delta k}{y} = \frac{\delta k}{k^\theta} = \delta k^{1 - \theta} = 0.25 = \theta
$$
This implies $\delta = r = 0.05$.

From the steady state version of the euler equation we found in problem 6, we have
$$
\frac{1}{\beta} = \theta \frac{y}{k} + 1 - \delta = \theta \frac{k^\theta}{k} + 1 - \delta = \theta \frac{1}{k^{1-\theta}} + 1 - \delta = \theta \frac{\delta}{\theta} + 1 - \delta = 1
$$
This implies $\beta = 1$.

We also can use this equation to note that
$$
\bar{k} = \left( \frac{\theta}{ \delta} \right)^{\frac{1}{1-\theta}}, \quad \bar{y} = \bar{k}^\theta, \quad \bar{c} = \bar{y} - \delta \bar{k}
$$

In [ ]:
δ = r = 0.05
β = 1
k_bar = (θ / δ)**(1 / (1 - θ))
y_bar = k_bar**θ
c_bar = y_bar - δ * k_bar

## Problem 7

The Markovian form of the system is
\begin{align*}
    E
    \begin{bmatrix}
    \tilde{c}_{t+1} \\
    \tilde{k}_{t+1} \\
    \tilde{z}_{t+1}
    \end{bmatrix}
    &=
    \begin{bmatrix}
    1 - \beta R \frac{c}{k} & R & \beta \frac{y}{k}(\rho +  R) \\
    -\frac{c}{k} & \frac{1}{\beta} & \frac{y}{k} \\
    0 & 0 & \rho
    \end{bmatrix}
    \begin{bmatrix}
    \tilde{c}_{t} \\
    \tilde{k}_{t} \\
    \tilde{z}_{t}
    \end{bmatrix} 
    =
    M
    \begin{bmatrix}
    \tilde{c}_{t} \\
    \tilde{k}_{t} \\
    \tilde{z}_{t}
    \end{bmatrix}  
\end{align*}

In [ ]:
R = (θ - 1) * (y_bar / k_bar)

M = np.array([[1 - β * R * (c_bar / k_bar),     R, β * (y_bar / k_bar) * (ρ + R)],
              [           -(c_bar / k_bar), 1 / β,                 y_bar / k_bar],
              [0,                               0,                             ρ]])

In [ ]:
λ, Γ = np.linalg.eig(M)  # Find eigenvalues and eigenvectors

In [ ]:
print(λ)

In our system we have one control/jump variable (consumption) and one unstable eigenvalue. Thus the Blanchard-Kahn condition is satisfied and we have a unique stationary solution to the linearized model.

## Problem 8

In [ ]:
Γ_inv = inv(Γ)
print(Γ_inv)

In [ ]:
# Partition the matrix based on stable and unstable eigenvalues
G_11 = Γ_inv[:1, :1]
G_12 = Γ_inv[:1, 1:]
G_22 = Γ_inv[1:, 1:]
G_21 = Γ_inv[1:, :1]

Construct the $z_t$ process

In [ ]:
np.random.seed(0)                         # Set random seed
T = 1200                                  # Length of time series
ε = np.random.normal(scale=σ, size=T)     # Draw random error terms
log_z = np.zeros(T)                       # To store z process
log_z[0] = 0

# Construct z process
for t in range(1, T):
    log_z[t] = ρ * log_z[t-1] + ε[t]
    
log_z = log_z[1:]  # Remove first observation

In [ ]:
z = np.exp(log_z)
plt.plot(z)
plt.title('Stochastic process $z_t$')
plt.show()

In [ ]:
c_tilde = np.zeros(T)
k_tilde = np.zeros(T)
y_tilde = np.zeros(T)

In [ ]:
# Construct series for log deviations of output, consumption and capital

for t in range(T-1):
    z_tilde = log_z[t]
    x = np.array([k_tilde[t],
                  z_tilde])
    c_tilde[t] = -inv(G_11) @ G_12 @ x
    y_tilde[t] = z_tilde + θ * k_tilde[t]
    k_tilde[t+1] = y_tilde[t] - c_tilde[t]

In [ ]:
discard_number = 200

# These are the log deviations from the steady state
log_k = k_tilde[discard_number:]
log_c = c_tilde[discard_number:]
log_y = y_tilde[discard_number:]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(np.exp(log_c), alpha=0.6, label='Consumption')
plt.plot(np.exp(log_y), alpha=0.6, label='Output')
plt.legend()
plt.show()

## Problem 9

In [ ]:
model = tsa.AR(log_c)
results = model.fit(maxlag=1)
results.params

The value of the AR(1) process is approximately 0.72

## Problem 10

We now have
$$
\frac{rk}{y} = \frac{rk}{k^\theta} = rk^{1 - \theta} = 0.65 = \theta
$$

To find $\delta$, note that the share of investment is
$$
\frac{i}{y} = \frac{\delta k}{y} = \frac{\delta k}{k^\theta} = \delta k^{1 - \theta} = 0.25
$$

Therefore,
$$
\frac{\delta k^{1 - \theta}}{r k^{1 - \theta}}
= \frac{\delta}{r} = \frac{0.25}{\theta}
$$
so
$$
\delta = \frac{0.25}{\theta} \cdot r
$$
We still have $\beta = 1$.

In [ ]:
# Find new parameters and steady state values
θ = 0.65
δ = (0.25 / θ) * r
k_bar = (θ / δ)**(1 / (1 - θ))
y_bar = k_bar**θ
c_bar = y_bar - δ * k_bar

# Find the Markovian matrix
R = (θ - 1) * (y_bar / k_bar)
M = np.array([[1 - β * R * (c_bar / k_bar),     R, β * (y_bar / k_bar) * (ρ + R)],
              [           -(c_bar / k_bar), 1 / β,                 y_bar / k_bar],
              [0,                               0,                             ρ]])

# Find eigenvalues and eigenvectors
λ, Γ = np.linalg.eig(M) 
Γ_inv = inv(Γ)
λ

In [ ]:
# Partition the matrix based on stable and unstable eigenvectors
G_11 = Γ_inv[:1, :1]
G_12 = Γ_inv[:1, 1:]
G_22 = Γ_inv[1:, 1:]
G_21 = Γ_inv[1:, :1]

In [ ]:
c_tilde = np.zeros(T)
k_tilde = np.zeros(T)
y_tilde = np.zeros(T)

In [ ]:
# Construct series for log deviations of output, consumption and capital

for t in range(T-1):
    z_tilde = log_z[t]
    x = np.array([k_tilde[t],
                  z_tilde])
    c_tilde[t] = -inv(G_11) @ G_12 @ x
    y_tilde[t] = z_tilde + θ * k_tilde[t]
    k_tilde[t+1] = y_tilde[t] - c_tilde[t]

In [ ]:
discard_number = 200

# These are the log deviations from the steady state
log_k = k_tilde[discard_number:]
log_c = c_tilde[discard_number:]
log_y = y_tilde[discard_number:]

In [ ]:
model = tsa.AR(log_c)
results = model.fit(maxlag=1)

results.params

This model is closer to a random walk for consumption (as indicated by the higher AR(1) parameter). 